In [1]:
!pip install -Uqqq pip --progress-bar off
!pip install -qqq torch==2.1 --progress-bar off
!pip install -qqq transformers==4.34.1 --progress-bar off
!pip install -qqq accelerate==0.23.0 --progress-bar off
!pip install -qqq bitsandbytes==0.41.1 --progress-bar off
!pip install -qqq llava-torch==1.1.1 --progress-bar off

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.20.1+cu124 requires torch==2.5.1, but you have torch 2.1.0 which is incompatible.
torchaudio 2.5.1+cu124 requires torch==2.5.1, but you have torch 2.1.0 which is incompatible.
pylibcugraph-cu12 24.12.0 requires pylibraft-cu12==24.12.*, but you have pylibraft-cu12 25.2.0 which is incompatible.
pylibcugraph-cu12 24.12.0 requires rmm-cu12==24.12.*, but you have rmm-cu12 25.2.0 which is incompatible.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 3.5.0 requires fsspec[http]<=2024.12.0,>=2023.1.0, but you have fsspec 2025.3.2 which is incompatible.
datasets 3.5.0 requires huggingface-hub>=0.24.0, but you have huggingface-hub 0.17.3 which is incompatible.
diffusers 0.32.2

In [2]:
import textwrap
from io import BytesIO

import requests
import torch
from llava.constants import DEFAULT_IMAGE_TOKEN, IMAGE_TOKEN_INDEX
from llava.conversation import SeparatorStyle, conv_templates
from llava.mm_utils import (
    KeywordsStoppingCriteria,
    get_model_name_from_path,
    process_images,
    tokenizer_image_token,
)
from llava.model.builder import load_pretrained_model
from llava.utils import disable_torch_init
from PIL import Image
disable_torch_init()
MODEL = "4bit/llava-v1.5-13b-3GB"
model_name = get_model_name_from_path(MODEL)
model_name
tokenizer, model, image_processor, context_len = load_pretrained_model(
                                                    model_path=MODEL, 
                                                    model_base=None, 
                                                    model_name=model_name, 
                                                    load_4bit=True )

[2025-05-19 00:53:13,797] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)


2025-05-19 00:53:17.760218: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747615997.960393      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747615998.018054      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Loading checkpoint shards:   0%|          | 0/9 [00:00<?, ?it/s]

In [3]:
CONV_MODE = "llava_v0"
def create_prompt(prompt: str):
    conv = conv_templates[CONV_MODE].copy()
    roles = conv.roles
    prompt = DEFAULT_IMAGE_TOKEN + "\n" + prompt
    conv.append_message(roles[0], prompt)
    conv.append_message(roles[1], None)
    return conv.get_prompt(), conv

prompt, _ = create_prompt("Describe the image")
print(prompt)

A chat between a curious human and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the human's questions.###Human: <image>
Describe the image###Assistant:


In [4]:
from tqdm import tqdm
import torch
from PIL import Image
import os
import zipfile

# —————————————————————————————————————————————
# Assumptions: image_processor, model, tokenizer,
# IMAGE_TOKEN_INDEX, create_prompt,
# tokenizer_image_token, SeparatorStyle,
# KeywordsStoppingCriteria are already defined and initialized.
# —————————————————————————————————————————————

# 1. Define the prompt
prompt_text = (
    "Please describe the image in detail, including the gender, "
    "physical features, skin color, emotional state (happy, sad, etc.), "
    "age group (child,mid age, young, elderly), clothing (including if wearing glasses or a hat), "
    "and background of the image."
)

# 2. Paths
image_dir     = '/kaggle/input/ffhq-1024x1024/images1024x1024'
check_folder  = '/kaggle/input/checkk/CHECK_6'   # where existing .txt 
output_dir    = '/kaggle/working/descriptions'
os.makedirs(output_dir, exist_ok=True)

# 3. Helper functions
def load_image(image_file):
    return Image.open(image_file).convert("RGB")

def process_image(image):
    args = {"image_aspect_ratio": "pad"}
    image_tensor = process_images([image], image_processor, args)
    return image_tensor.to(model.device, dtype=torch.float16)

def ask_image(image: Image, prompt: str):
    image_tensor = process_image(image)
    prompt, conv = create_prompt(prompt)
    input_ids = (
        tokenizer_image_token(prompt, tokenizer, IMAGE_TOKEN_INDEX, return_tensors="pt")
        .unsqueeze(0)
        .to(model.device)
    )
    stop_str = conv.sep if conv.sep_style != SeparatorStyle.TWO else conv.sep2
    stopping_criteria = KeywordsStoppingCriteria(
        keywords=[stop_str], tokenizer=tokenizer, input_ids=input_ids
    )
    with torch.inference_mode():
        output_ids = model.generate(
            input_ids,
            images=image_tensor,
            do_sample=True,
            temperature=0.01,
            max_new_tokens=512,
            use_cache=True,
            stopping_criteria=[stopping_criteria],
        )
    return tokenizer.decode(
        output_ids[0, input_ids.shape[1] :], skip_special_tokens=True
    ).strip()

# 4. Build the set of already-processed .txt filenames
original = set(os.listdir(check_folder))
generated = set(os.listdir(output_dir))
existing_files = original | generated

# 5. Process images 20000 to 24500
image_files = os.listdir(image_dir)[31000:35500]

for image_file in tqdm(image_files, desc="Processing images"):
    txt_name = os.path.splitext(image_file)[0] + '.txt'
    if txt_name in existing_files:
        continue  # skip if already in check_folder or already generated

    image_path = os.path.join(image_dir, image_file)
    image = load_image(image_path)

    description = ask_image(image, prompt_text)

    # save to output_dir
    with open(os.path.join(output_dir, txt_name), 'w', encoding='utf-8') as f:
        f.write(description)

    existing_files.add(txt_name)

# 6. Compress results
zip_path = '/kaggle/working/descriptions_31k_to_35500k.zip'
with zipfile.ZipFile(zip_path, 'w') as zipf:
    for fname in os.listdir(output_dir):
        zipf.write(os.path.join(output_dir, fname), arcname=fname)

print(f"Done! ZIP archive at: {zip_path}")


Processing images: 100%|██████████| 4500/4500 [10:35:50<00:00,  8.48s/it]


Done! ZIP archive at: /kaggle/working/descriptions_31k_to_35500k.zip
